KNN for WB

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report


df = pd.read_csv('feature_file_1740153842.csv')


wristband_columns = [col for col in df.columns if col.startswith(('bvp', 'gsr', 'tmp'))]
wristband_columns.append('emotion')  # Add emotion column as labels
df_wristband = df[wristband_columns]


label_encoder = LabelEncoder()
df_wristband['emotion'] = label_encoder.fit_transform(df_wristband['emotion'])


X_wristband = df_wristband.drop(columns=['emotion'])
y_wristband = df_wristband['emotion']


smote = SMOTE(random_state=42)
X_resampled_wristband, y_resampled_wristband = smote.fit_resample(X_wristband, y_wristband)


scaler = StandardScaler()
X_resampled_wristband = scaler.fit_transform(X_resampled_wristband)


X_train_wristband, X_test_wristband, y_train_wristband, y_test_wristband = train_test_split(
    X_resampled_wristband, y_resampled_wristband, test_size=0.2, random_state=42
)


knn_wristband = KNeighborsClassifier(n_neighbors=5)
knn_wristband.fit(X_train_wristband, y_train_wristband)


y_pred_wristband = knn_wristband.predict(X_test_wristband)


accuracy_wristband = np.mean(y_pred_wristband == y_test_wristband)
print(f"Wristband Test Accuracy: {accuracy_wristband:.4f}")


conf_matrix_wristband = confusion_matrix(y_test_wristband, y_pred_wristband)
conf_matrix_df_wristband = pd.DataFrame(
    conf_matrix_wristband,
    index=label_encoder.classes_,
    columns=label_encoder.classes_
)
print("Wristband Confusion Matrix:")
print(conf_matrix_df_wristband)


print("Wristband Classification Report:")
print(classification_report(y_test_wristband, y_pred_wristband, target_names=label_encoder.classes_))

C:\Users\Abdur Rafay\AppData\Local\Temp\ipykernel_25944\3147597981.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wristband['emotion'] = label_encoder.fit_transform(df_wristband['emotion'])


Wristband Test Accuracy: 0.4606
Wristband Confusion Matrix:
         Excited  Fear  Happy  Sad  relax
Excited       51     6      5    5      6
Fear          10    35     13    3      4
Happy          7    15     31    2      6
Sad           19    12     14   16      8
relax         18     8     11   13     25
Wristband Classification Report:
              precision    recall  f1-score   support

     Excited       0.49      0.70      0.57        73
        Fear       0.46      0.54      0.50        65
       Happy       0.42      0.51      0.46        61
         Sad       0.41      0.23      0.30        69
       relax       0.51      0.33      0.40        75

    accuracy                           0.46       343
   macro avg       0.46      0.46      0.45       343
weighted avg       0.46      0.46      0.45       343



KNN for EEG

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
import random
import matplotlib.pyplot as plt
import seaborn as sns


seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)


df = pd.read_csv('feature_file_1740153842.csv')


eeg_columns = [col for col in df.columns if not col.startswith(('bvp', 'gsr', 'tmp'))]
df_eeg = df[eeg_columns]


label_encoder = LabelEncoder()
df_eeg['emotion'] = label_encoder.fit_transform(df_eeg['emotion'])


X_eeg = df_eeg.drop(columns=['emotion'])
y_eeg = df_eeg['emotion']


def augment_data(X, y, noise_factor=0.1, num_augmented_per_class=500):
    np.random.seed(seed_value)
    X_augmented = []
    y_augmented = []
    
    
    class_counts = np.bincount(y)

  
    for class_index in range(len(class_counts)):
        num_to_generate = max(0, num_augmented_per_class - class_counts[class_index])
        for _ in range(num_to_generate):
          
            idx = np.random.choice(np.where(y == class_index)[0])
            noise = noise_factor * np.random.randn(*X[idx].shape)
            X_augmented.append(X[idx] + noise)
            y_augmented.append(y[idx])
    
    return np.array(X_augmented), np.array(y_augmented)


X_augmented, y_augmented = augment_data(X_eeg.values, y_eeg.values)


X_combined = np.vstack((X_eeg.values, X_augmented))
y_combined = np.concatenate((y_eeg.values, y_augmented))


X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y_combined, test_size=0.2, random_state=seed_value, stratify=y_combined
)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


knn_model = KNeighborsClassifier(n_neighbors=5)


knn_model.fit(X_train, y_train)


y_pred = knn_model.predict(X_test)


train_accuracy = accuracy_score(y_train, knn_model.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)


print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)

C:\Users\Abdur Rafay\AppData\Local\Temp\ipykernel_25944\4272720666.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eeg['emotion'] = label_encoder.fit_transform(df_eeg['emotion'])


Training Accuracy: 0.8010
Test Accuracy: 0.6460

Classification Report:
               precision    recall  f1-score   support

     Excited       0.68      0.65      0.67       100
        Fear       0.67      0.76      0.71       100
       Happy       0.62      0.69      0.65       100
         Sad       0.52      0.66      0.58       100
       relax       0.89      0.47      0.61       100

    accuracy                           0.65       500
   macro avg       0.68      0.65      0.65       500
weighted avg       0.68      0.65      0.65       500


Confusion Matrix:
 [[65 10 12 12  1]
 [ 9 76  5  7  3]
 [ 2  6 69 23  0]
 [ 5 11 16 66  2]
 [14 10  9 20 47]]


KNN for Combined classification

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import class_weight
import random
import matplotlib.pyplot as plt
import seaborn as sns


seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)


df = pd.read_csv('feature_file_1740153842.csv')


eeg_columns = [col for col in df.columns if not col.startswith(('bvp', 'gsr', 'tmp'))]
wristband_columns = [col for col in df.columns if col.startswith(('bvp', 'gsr', 'tmp'))]
combined_columns = eeg_columns + wristband_columns
df_combined = df[combined_columns]

label_encoder = LabelEncoder()
df_combined['emotion'] = label_encoder.fit_transform(df_combined['emotion'])


X_combined = df_combined.drop(columns=['emotion'])
y_combined = df_combined['emotion']


def augment_data(X, y, noise_factor=0.1, num_augmented_per_class=500):
    np.random.seed(seed_value)
    X_augmented = []
    y_augmented = []
    
    
    class_counts = np.bincount(y)

    
    for class_index in range(len(class_counts)):
        num_to_generate = max(0, num_augmented_per_class - class_counts[class_index])
        for _ in range(num_to_generate):
            
            idx = np.random.choice(np.where(y == class_index)[0])
            noise = noise_factor * np.random.randn(*X[idx].shape)
            X_augmented.append(X[idx] + noise)
            y_augmented.append(y[idx])
    
    return np.array(X_augmented), np.array(y_augmented)


X_augmented, y_augmented = augment_data(X_combined.values, y_combined.values)


X_combined_augmented = np.vstack((X_combined.values, X_augmented))
y_combined_augmented = np.concatenate((y_combined.values, y_augmented))


X_train, X_test, y_train, y_test = train_test_split(
    X_combined_augmented, y_combined_augmented, test_size=0.2, random_state=seed_value, stratify=y_combined_augmented
)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


knn_model = KNeighborsClassifier(n_neighbors=5)


knn_model.fit(X_train, y_train)


y_pred = knn_model.predict(X_test)


train_accuracy = accuracy_score(y_train, knn_model.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)


print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)

Training Accuracy: 0.8215
Test Accuracy: 0.6660

Classification Report:
               precision    recall  f1-score   support

     Excited       0.72      0.69      0.70       100
        Fear       0.67      0.78      0.72       100
       Happy       0.61      0.65      0.63       100
         Sad       0.57      0.67      0.61       100
       relax       0.86      0.54      0.66       100

    accuracy                           0.67       500
   macro avg       0.68      0.67      0.67       500
weighted avg       0.68      0.67      0.67       500


Confusion Matrix:
 [[69 11 13  4  3]
 [ 3 78  8  9  2]
 [ 3  8 65 24  0]
 [10  9 10 67  4]
 [11 11 10 14 54]]
